In [1]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

transformers.logging.set_verbosity_error()
import torch

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

llm = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct", dtype=dtype, device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
tokenizer.pad_token = tokenizer.eos_token

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [ ]:
v = torch.load("steering_vectors.pt", map_location=device)[(15, 21576)].to(device, dtype=dtype)

layer_idx = 15
coefficient = 4.0


def steering_hook(module, input, output):
    return output + coefficient * v


# register
hook_handle = llm.model.layers[layer_idx].mlp.register_forward_hook(steering_hook)

In [3]:
messages = [{"role": "user", "content": "Give me some ideas for starting a new business"}]
input_ids = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_tensors="pt", return_dict=True
).to(device)
input_ids = input_ids.to("mps")
output_ids = llm.generate(**input_ids, max_new_tokens=128, do_sample=False, repetition_penalty=1.1)
answer = output_ids.tolist()[0][input_ids.input_ids.shape[1] :]
print(tokenizer.decode(answer, skip_special_tokens=True))

Here are some business ideas across various industries:

**E-commerce and Online Businesses**

1. **Niche online store**: Create an e-commerce platform selling products related to a specific niche, such as outdoor gear or pet accessories.
2. **Subscription box service**: Offer monthly deliveries of curated products, like gourmet food, beauty samples, or children's toys.
3. **Digital marketing agency**: Help small businesses with social media management, SEO, and content creation.
4. **Online education platform**: Develop courses or tutorials on topics like coding, photography, or language learning.

**Food and Beverage**

1. **Gourmet food truck**: Serve


In [ ]:
input_ids = llm.tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True)
with llm.generate() as tracer:
    with tracer.invoke(input_ids):
        with tracer.all() as idx:
            for sc in steering_components:
                layer, strength, vector = sc["layer"], sc["strength"], sc["vector"]
                length = llm.model.layers[layer].output.shape[1]
                amount = (strength * vector).unsqueeze(0).expand(length, -1).unsqueeze(0).clone()
                llm.model.layers[layer].output += amount
    with tracer.invoke():
        trace = llm.generator.output.save()

answer = llm.tokenizer.decode(trace[0][len(input_ids):], skip_special_tokens=True)